In [1]:
import os
import sys
import copy
from typing import Any, Dict, List
from IPython.display import display
from IPython.display import Markdown
from autogen import AfterWork, OnCondition, AfterWorkOption, SwarmAgent, SwarmResult, initiate_swarm_chat, UpdateSystemMessage, ConversableAgent, register_hand_off
from cmbagent import CMBAgent
cmbagent = CMBAgent(agent_list = ['classy_sz'],#['classy_sz','act'],#['classy_sz'],
                    skip_memory = True,
                    skip_rag_software_formatter = False,
                    verbose=False,
                    agent_type='swarm',
                    agent_llm_configs = {
                    'engineer': {
                        # Choose your model.
                        "model": "claude-3-5-sonnet-20241022",
                        # You need to provide your API key here.
                        "api_key": os.getenv("ANTHROPIC_API_KEY"),
                        "api_type": "anthropic",
                        }
                    },
                    # make_vector_stores = ['classy_sz']
                    agent_instructions = {
'planner' :  
"""
  You are the planner agent in the team. 

  #### Main purpose

  Suggest plan to solve the main task.

""",
                        
'classy_sz_agent':
"""
You are a Retrieval-Augmented Generation (RAG) agent tasked with performing file searches and suggesting Python code snippets to help setting-up computations with the classy_sz package.

You must use tool call (i.e., file search).

Your answer must have the following structure:

**File Search Results:**

{retrieval task details} 

**Files Consulted:**

{files}

**Code Explanation:**

{code_explanation}

**Python Code:**

{python_code}


""",

"engineer":
"""
You are an expert Python software developer.

Review codes previously obtained and edit it into a single self-consistent Python block.

      
  **IMPORTANT**: 
  
    - Return one and only one Python code block in your response.
    - Focus on one step at a time.
    - Do not suggest incomplete code.
    - Do not produce code blocks that are not intended for execution.
    - Include only one code block per response.
    - When a plot is requested, it must be saved into a png file. 
    - Review existing code before writing new code.
    - Write Python to solve tasks within the code block.
    - Avoid creating ".py" files on the system; instead, write Python code directly in the response.
    - Do not check for installed packages.
    - Do not install new packages.
    - Do not use pip.
    - Do not use '.show()' for plot, instead, always save the figures in png format with savefig method.
  
    **Use Python language only.** 

After producing code, you check with admin for code approval.
  
""",
"rag_software_formatter":
"""
You are a formatting agent to convert responses from RAG agents into a structured output.
""",
"executor":
"""
You execute python code provided to you by the engineer.
""",
                        
                    }
                   )

cmbagent.clear_cache()
# task = "Compute temperature cls using standard cosmological lcdm values as measured by ACT DR6 lensing analysis."
# cmbagent.solve(task)


task = "Compute CMB temperature power spectra with classy_sz. Do not plot polarization power spectra."

llm_config = cmbagent.llm_config#{"model": "gpt-4o-mini", "cache_seed": None}

CMBAgent version: 0.0.1post2
agent_type:  swarm
Checking agent: classy_sz_agent
Agent classy_sz_agent exists in available assistants with id: asst_IEFG6lIlkKxuq8h72q6UGsf9
	- admin
	- planner
	- engineer
	- executor
	- rag_software_formatter
	- classy_sz_agent
setting agent:  classy_sz_agent

 adding files to instructions:  ['set_classy_sz_in_cobaya.md', 'classy_sz_cmb_cls-bkp.md', 'classy_sz_matter_pk_nonlinear.md', 'classy_sz_parameters_and_derived_parameters.md']


In [28]:
# os.getenv("ANTHROPIC_API_KEY")
# see notebooks/bbolliet_local/cmbagent.ipynb for an example on how to setup anthropic

In [2]:
from cmbagent.structured_output import PlannerResponse
from cmbagent.structured_output import PlanReviewerResponse

from typing import Any, Dict
from pydantic import BaseModel, Field

# Structured Output agent, formatting the itinerary into a structured format through the response_format on the LLM Configuration
plan_reviewer_structured_config_list = copy.deepcopy(cmbagent.llm_config)
plan_reviewer_structured_config_list['config_list'][0]['response_format'] = PlanReviewerResponse
# for config in structured_config_list:
#     config["response_format"] = PlanReviewerResponse
# print(plan_reviewer_structured_config_list)

class StructuredCode(BaseModel):
    python_code: str = Field(..., description="The Python code in a form ready to execute")

structured_code_agent_config_list = copy.deepcopy(cmbagent.llm_config)
structured_code_agent_config_list['config_list'][0]['response_format'] = StructuredCode

In [3]:
# cmbagent.planner.llm_config

In [4]:
cmbagent.planner.info

{'name': 'planner',
 'instructions': '\n  You are the planner agent in the team. \n\n  #### Main purpose\n\n  Suggest plan to solve the main task.\n\n',
 'description': 'This is the planner agent: the first agent to speak. \nAn agent that designs a PLAN. \nIt can be asked to:\n-design a PLAN\n-revise the PLAN before approval\n-modify the PLAN before approval\n-update the PLAN before approval\n',
 'allowed_transitions': ['admin']}

In [5]:
cmbagent.classy_sz.info

{'name': 'classy_sz_agent',
 'instructions': "\nYou are a Retrieval-Augmented Generation (RAG) agent tasked with performing file searches and suggesting Python code snippets to help setting-up computations with the classy_sz package.\n\nYou must use tool call (i.e., file search).\n\nYour answer must have the following structure:\n\n**File Search Results:**\n\n{retrieval task details} \n\n**Files Consulted:**\n\n{files}\n\n**Code Explanation:**\n\n{code_explanation}\n\n**Python Code:**\n\n{python_code}\n\n\n\n You have access to the following files: ['set_classy_sz_in_cobaya.md', 'classy_sz_cmb_cls-bkp.md', 'classy_sz_matter_pk_nonlinear.md', 'classy_sz_parameters_and_derived_parameters.md'].\n",
 'assistant_config': {'assistant_id': 'asst_IEFG6lIlkKxuq8h72q6UGsf9',
  'tools': [{'type': 'file_search'}],
  'tool_resources': {'file_search': {'vector_store_ids': ['vs_8kqbw7TnNaHGKNj9vA9JSxvo']}},
  'temperature': 1e-05,
  'top_p': 0.05},
 'description': 'This is the classy_sz_agent: a retr

In [6]:
cmbagent.classy_sz.agent

In [7]:
# cmbagent.classy_sz.agent.llm_config

In [8]:
cmbagent.rag_software_formatter.info

{'name': 'rag_software_formatter',
 'instructions': '\nYou are a formatting agent to convert responses from RAG agents into a structured output.\n',
 'description': 'This is the rag_software_formatter agent.\nIt formats the data retrieved by other agents into a specific JSON-structured output. The key output is a Python code block.\n',
 'allowed_transitions': ['admin']}

In [9]:
cmbagent.executor.info

{'name': 'executor',
 'human_input_mode': 'NEVER',
 'max_consecutive_auto_reply': 50,
 'instructions': '\nYou execute python code provided to you by the engineer.\n',
 'description': 'An executor agent that executes python code written by the engineer.\nThe executor does not write code. \nThe executor does not suggest code.\nThe executor runs code.\n',
 'timeout': 3600,
 'allowed_transitions': ['admin']}

In [10]:
cmbagent.admin.agent.name

'admin'

In [11]:
cmbagent.clear_cache()
## availablility conditions 
# available (Union[Callable, str]): Optional condition to determine if this ON_CONDITION is available. Can be a Callable or a string.
#     If a string, it will look up the value of the context variable with that name, which should be a bool.

def is_plan_approved_in_context(agent: SwarmAgent, messages: List[Dict[str, Any]]) -> bool:
    is_plan_approved = agent.get_context("plan_approved")
    # print("is_plan_approved:",is_plan_approved)
    return is_plan_approved






plan_reviewer_message = """You are a plan reviewer.
You provide recommendations to improve to the plan suggested by the planner, 
consisting of recommendations to make sure that sub-task are well-distrubted between all agents and that at the end of the plan, 
some code to solve the task is proposed. 
A good plan must:
- split the main task into elementary sub-tasks.
- each sub-task must be carried out by a single agent.
- there should not be repetitions 
- code is executed at the very end only
- a plot must be produced at the end of the task
Your response should be concise and specific.


Check that the agents called in each sub-task only include: 

- classy_sz_agent: retrieves information on how to compute theoretical prediction for CMB/LSS observables with the classy_sz code.
- engineer: an expert Python coder who writes entire pipelines.
- executor: it executes the Python code provided by the engineer. 

Also, make sure that according to the plan, the engineer agent should be called after all relevant information has been collected from the other agents in the workflow. 

Each new review you provide must differ from the previous ones, providing increasing level of details as the reviewing process progresses.
"""




plan_reviewer = ConversableAgent(
    name="plan_reviewer", 
    llm_config=plan_reviewer_structured_config_list, 
    system_message=plan_reviewer_message, 

)

# 1. Context
shared_context = {
    "plans": [],
    "reviews": [],
    # Will be decremented, resulting in 0 (aka False) when no reviews are left
    "reviews_left": 2,
    "review_recorded": False,
    "plan_recorded": False,
    "reviews_left_and_plan_recorded" : False,

    # plan approval
    "plan_approved": False,

    # classy_sz workflow
    # "message_for_classy_sz_agent" : None,
    # "has_classy_sz_agent_message" : False,
    # code approval
    "code_approved": False,
    "structured_code": None,

}



# 2. Functions

def mark_code_as_approved(context_variables: Dict[str, Any]) -> SwarmResult:
    """To record the code approval from the Admin, after Admin has approved."""
    context_variables["code_approved"] = True
    # Returning the updated context so the shared context can be updated
    return SwarmResult(#agent=cmbagent.executor.agent,
                       # agent=AfterWorkOption.REVERT_TO_USER,## transfer to user
                       agent=structured_code_agent,
                       values="Code is approved....",
                       context_variables=context_variables)


def format_structured_code(context_variables: Dict[str, Any], structured_code: str) -> SwarmResult:
    """Once a code is approved, format it and pass on to executor agent."""
    # Ensure the code is approved, if not, back to the plan manager agent to confirm it with the admin
    if not context_variables["code_approved"]:
        return SwarmResult(
            agent= plan_implementer,
            values="Code is not approved, you must call Admin and ask for approval first.",
        )

    # print("\n\n structured_code......",structured_code)
    # context_variables["structured_code"].append(structured_code)
    # Ensure 'structured_code' is a list before appending
    if context_variables.get("structured_code") is None:
        context_variables["structured_code"] = []

    context_variables["structured_code"].append(structured_code)

    # This will update the context variables and then transfer to the Route Timing agent
    return SwarmResult(
        agent=cmbagent.executor.agent, 
        context_variables=context_variables, 
        values="Structured code stored and approved. Executing..."
    )


def approve_plan(context_variables: dict)->SwarmResult:
    """Approve the Plan."""
    context_variables["plan_approved"] = True
    # Returning the updated context so the shared context can be updated
    return SwarmResult(agent=plan_implementer, ## transfer to plan implementer
                       values="Plan is approved. Starting implementation...",
                       context_variables=context_variables)

def record_plan(plan_suggestion: str, context_variables: dict) -> SwarmResult:
    """Record Plan."""
    context_variables["plans"].append(plan_suggestion)
    context_variables["review_recorded"] = False
    context_variables["plan_recorded"] = True
    context_variables["reviews_left_and_plan_recorded"] = context_variables["plan_recorded"]*context_variables["reviews_left"]

    # Returning the updated context so the shared context can be updated
    if context_variables["reviews_left"] == 0:
        context_variables["review_recorded"] = True
        return SwarmResult(agent=plan_manager,
                           values="Final plan has been logged. We can proceed with approval.",
                           context_variables=context_variables)
    else:
        return SwarmResult(agent=plan_reviewer, ## transfer to plan reviewer
                           values="Plan has been logged. Next, review it and provide recommendations for improvements.",
                           context_variables=context_variables)


# cmbagent.planner.agent.add_single_function(suggest_record_plan)



def record_review(plan_review: str, context_variables: dict) -> SwarmResult:
    """ Record Plan reviews."""
    context_variables["reviews"].append(plan_review)
    context_variables["review_recorded"] = True
    context_variables["reviews_left"] -= 1
    context_variables["plan_recorded"] = False
    context_variables["reviews_left_and_plan_recorded"] = context_variables["plan_recorded"]*context_variables["reviews_left"]

    # Controlling the flow to the next agent from a tool call
    if context_variables["reviews_left"] < 0:
        context_variables["plan_recorded"] = True
        return SwarmResult(agent=plan_manager,
                           values="No further recommendations to be made on the plan. Update plan and proceed",
                           context_variables=context_variables)
    else:
        return SwarmResult(agent=cmbagent.planner.agent,  ## transfer back to planner
                           values="Recommendations logged. Next, provide an updated version of plan.",
                           context_variables=context_variables)

# plan_reviewer.add_single_function(suggest_record_review)


# def record_message_for_classy_sz_agent(message_for_classy_sz_agent: str, context_variables: dict) -> SwarmResult:
#     """ Log the message for the classy_sz_agent. """
#     context_variables["has_classy_sz_agent_message"] = True
#     context_variables["message_for_classy_sz_agent"] = message_for_classy_sz_agent
#     return SwarmResult(agent=plan_implementer,
#                        values="The message for the classy_sz_agent has been logged. Relevant information can now be retrieved with the classy_sz_agent.",
#                        context_variables=context_variables)


plan_manager_message = """
You are an plan management agent that manages the workflow in the designing of the plan.

All versions of the plan suggested must be logged.

All reviews made must be logged.

After each review, the plan must be updated to a new version, implementing carefully the recommendations.

Once plan has been proposed, reviewed and updated as many times as needed, and all plan versions and reviews are recorded, you may approve the plan.  

The current status of this workflow is:
- Plan version recorded? {plan_recorded}
- Plan review recorded? {review_recorded}
- Plan approved? {plan_approved}

You call one tool at a time. 
"""



def show_final_plan(context_variables: dict) -> str:
    """Call this function to show the final plan before sending for approval."""
    # plan = context_variables["plans"]
    # display(Markdown(plan[-1]))
    # Get the latest plan from the list.
    try:
        # Retrieve the JSON string representing the plan
        plan_json = context_variables["plans"][-1]
        
        # Parse the JSON string into a PlannerResponse object using Pydantic
        plan = PlannerResponse.model_validate_json(plan_json)

        formatted_plan = plan.format()
        # Display the formatted plan as Markdown.
        display(Markdown(formatted_plan))

    except:
        print(f"could not display final plan in markdown format. The raw plan is:\n{context_variables["plans"][-1]}\n")
        # sys.exit()
    return """Plan ready for approval."""


plan_implementer_message =  """
You manage the workflow for implementing the plan.
You distribute sub-tasks to agents and collect results. 

You perform one step at a time. 
Never call more than one tool at a time. 

When distributing sub-task, be very specific on what is needed for each sub-task.


Finally, once a code has been provided, YOU MUST ask the Admin if they are happy with the code.
If Admin approves the code, you mark the code as approved.
"""
# Before retrieving information with the classy_sz agent, 
# you must create the message for the classy_sz_agent explaining in details what is needed from it in this sub-task of the plan. This message must be logged.




plan_manager = ConversableAgent(
    name="plan_manager", 
    update_agent_state_before_reply=[
        UpdateSystemMessage(plan_manager_message),
    ],
    functions=[record_plan, record_review, show_final_plan, approve_plan],
    llm_config=llm_config,
)


plan_implementer = ConversableAgent(
    name="plan_implementer", 
    system_message=plan_implementer_message, 
    # functions=[record_message_for_classy_sz_agent],
    functions=[mark_code_as_approved],
    llm_config=llm_config,
)

structured_code_agent = ConversableAgent(
    name="structured_code_agent",
    system_message="You are a code formatting agent, you format the provided Python code provided. You must use your tool call.",
    llm_config=structured_code_agent_config_list,
    functions=[format_structured_code],
)


register_hand_off(
    agent = plan_manager,
    hand_to = 
    [
        OnCondition( 
            # condition (str): 
            # The condition for transitioning to the target agent, 
            # evaluated by the LLM to determine whether to call the underlying function/tool which does the transition.
            target=cmbagent.planner.agent, 
            condition="Provide new version of the plan based on recommendations.",
            available="reviews_left"
        ),
        OnCondition(
            target=plan_reviewer, 
            condition="Review the plan and provide recommendations.",
            available="reviews_left"
        ),
        OnCondition(
            target=plan_implementer,
            condition="Implement the plan.",
            available=is_plan_approved_in_context,
        ),
        AfterWork(AfterWorkOption.REVERT_TO_USER), # Handles the next step in the conversation when an agent doesn't suggest a tool call or a handoff
        # AFTER_WORK(plan_manager)
    ]
)


register_hand_off(
    agent = plan_reviewer,
    hand_to = [
        # ON_CONDITION( 
        #     # condition (str): 
        #     # The condition for transitioning to the target agent, 
        #     # evaluated by the LLM to determine whether to call the underlying function/tool which does the transition.
        #     target=plan_manager, 
        #     condition="Update the plan version. DO NOT call more than once at a time",
        #     # available="review_recorded"
        # ),
        OnCondition(
            target=plan_manager, 
            condition="Review made. Now, record the review."
        ),
        # AFTER_WORK(AfterWorkOption.REVERT_TO_USER), # Handles the next step in the conversation when an agent doesn't suggest a tool call or a handoff
        AfterWork(plan_manager)
    ]
)



register_hand_off(
    agent = cmbagent.planner.agent,
    hand_to = [
        # ON_CONDITION(
        #     target=plan_reviewer,
        #     condition="Review this version of the plan. DO NOT call more than once at a time",
        #     available="reviews_left_and_plan_recorded",
        #     #### available (Union[Callable, str]): Optional condition to determine if this ON_CONDITION is available. Can be a Callable or a string.
        #     ### If a string, it will look up the value of the context variable with that name, which should be a bool.
        # ),

        OnCondition(
            target=plan_manager, 
            condition="Plan made. Now, record the plan.",
        ),
        AfterWork(plan_manager),
    ]
)

# def extract_message_for_classy_sz_agent_from_context(recipient: ConversableAgent, messages, sender: ConversableAgent, config):
#     """Extract the message for classy_sz_agent from the context"""
#     message_for_classy_sz_agent = sender.get_context("message_for_classy_sz_agent")
#     if message_for_classy_sz is None:
#         return f"Message for classy_sz has not been logged. Please log it."
#     else:
#         return f"{message_for_classy_sz_agent}"




nested_chat_classy_sz = {
    # "carryover_config": {"summary_method": "reflection_with_llm"},
    "carryover_config": {"summary_method": "all"},
    "recipient": cmbagent.classy_sz.agent,
    # "message": extract_message_for_classy_sz_agent_from_context,
    # "message": "Question to ask the classy_sz_agent regarding the information we need at this stage in the plan.",
    "max_turns": 1,
    "summary_method": "last_msg",
}


nested_chat_formatter = {
    "recipient": cmbagent.rag_software_formatter.agent,
    "message": "Format the answer provided by a RAG agent.",
    "max_turns": 1,
    "summary_method": "last_msg",
}

chat_queue = [nested_chat_classy_sz,nested_chat_formatter]


# def has_classy_sz_message_in_context(agent: SwarmAgent, messages: List[Dict[str, Any]]) -> bool:
#     return agent.get_context("has_classy_sz_message")

register_hand_off(
    agent = plan_implementer,
    hand_to = [
        OnCondition( 
            # condition (str): 
            # The condition for transitioning to the target agent, 
            # evaluated by the LLM to determine whether to call the underlying function/tool which does the transition.
            target=cmbagent.engineer.agent, 
            condition="Review code and edit it into a single self-consistent Python code block.",
            # available="review_recorded"
        ),

        OnCondition( 
            # condition (str): 
            # The condition for transitioning to the target agent, 
            # evaluated by the LLM to determine whether to call the underlying function/tool which does the transition.
            target=structured_code_agent, 
            condition="Code approved by Admin.",
            # available="review_recorded"
        ),
        # ON_CONDITION( 
        #     # condition (str): 
        #     # The condition for transitioning to the target agent, 
        #     # evaluated by the LLM to determine whether to call the underlying function/tool which does the transition.
        #     target=cmbagent.executor.agent, 
        #     condition="The code has been approved, execute it.",
        #     available="code_approved"
        # ),
        
        OnCondition(
            target={
                "chat_queue": chat_queue,
            },
            condition="Retrieve information on how to use classy_sz with the classy_sz agent.",
            # available=has_classy_sz_message_in_context,
        ),


        AfterWork(AfterWorkOption.REVERT_TO_USER), # Handles the next step in the conversation when an agent doesn't suggest a tool call or a handoff
        # AFTER_WORK(plan_manager)
    ]
)

register_hand_off(
    agent = cmbagent.executor.agent,
    hand_to = [
        # ON_CONDITION(
        #     target=plan_reviewer,
        #     condition="Review this version of the plan. DO NOT call more than once at a time",
        #     available="reviews_left_and_plan_recorded",
        #     #### available (Union[Callable, str]): Optional condition to determine if this ON_CONDITION is available. Can be a Callable or a string.
        #     ### If a string, it will look up the value of the context variable with that name, which should be a bool.
        # ),

        # ON_CONDITION(
        #     target=plan_implementer, 
        #     condition="Code executed successfully.",
        # ),
        AfterWork(plan_implementer),
    ]
)


#### classy_sz agent is inside a nested chat 
# cmbagent.classy_sz.agent.register_hand_off(
#     [
#         # ON_CONDITION(
#         #     target=plan_reviewer,
#         #     condition="Review this version of the plan. DO NOT call more than once at a time",
#         #     available="reviews_left_and_plan_recorded",
#         #     #### available (Union[Callable, str]): Optional condition to determine if this ON_CONDITION is available. Can be a Callable or a string.
#         #     ### If a string, it will look up the value of the context variable with that name, which should be a bool.
#         # ),

#         # ON_CONDITION(
#         #     target=cmbagent.classy_sz.agent, 
#         #     condition="Retrieve information on how to use classy_sz.",
#         # ),
#         # AFTER_WORK(plan_implementer),
#         AFTER_WORK(AfterWorkOption.TERMINATE)
#     ]
# )


register_hand_off(
    agent = cmbagent.engineer.agent,
    hand_to = [
        # ON_CONDITION(
        #     target=plan_reviewer,
        #     condition="Review this version of the plan. DO NOT call more than once at a time",
        #     available="reviews_left_and_plan_recorded",
        #     #### available (Union[Callable, str]): Optional condition to determine if this ON_CONDITION is available. Can be a Callable or a string.
        #     ### If a string, it will look up the value of the context variable with that name, which should be a bool.
        # ),

        # ON_CONDITION(
        #     target=cmbagent.executor.agent, 
        #     condition="Code is approved.",
        #     available="code_approved",
        # ),
        AfterWork(plan_implementer),
        # AFTER_WORK(AfterWorkOption.TERMINATE)
        # AFTER_WORK(AfterWorkOption.REVERT_TO_USER),
    ]
)

# cmbagent.engineer.agent.add_single_function(log_code_approval)


register_hand_off(
    agent = structured_code_agent,
    hand_to=[AfterWork(cmbagent.executor.agent)])

# @export_module("autogen")
# AFTER_WORK:
# class AfterWorkOption(Enum):
#     TERMINATE = "TERMINATE"
#     REVERT_TO_USER = "REVERT_TO_USER"
#     STAY = "STAY"
#     SWARM_MANAGER = "SWARM_MANAGER"


# order_summariser_agent = ConversableAgent(
#     name="order_summariser_agent",
#     system_message="You are an order summariser agent that provides a summary of the order details.",
#     llm_config=llm_config,
# )


# 5. Run the Swarm which returns the chat and updated context variables
chat_result, context_variables, last_agent = initiate_swarm_chat(
    initial_agent=cmbagent.planner.agent,
    agents=[cmbagent.planner.agent, 
            plan_reviewer,plan_manager,
            plan_implementer,
            # order_summariser_agent,#cmbagent.classy_sz.agent,
            structured_code_agent,
            cmbagent.engineer.agent,
            cmbagent.executor.agent],
    messages=task,
    user_agent=cmbagent.admin.agent,
    context_variables=shared_context,
    max_rounds = 100,
)

print(f"Number of reviews: {len(context_variables['plans'])}")
print(f"Reviews remaining: {context_variables['reviews_left']}")
print(f"Final Plan for the task:\n{context_variables['plans'][-1]}")


Message from admin:



Compute CMB temperature power spectra with classy_sz. Do not plot polarization power spectra.


--------------------------------------------------------------------------------

Calling: planner...


>>>>>>>> USING AUTO REPLY...


Model,agent,Cost,Prompt Tokens,Completion Tokens,Total Tokens
gpt-4o-2024-08-06,planner,0.00580,89,558,647


Message from planner:



...

Message from plan_implementer:



The execution of the code was successful, and the CMB temperature power spectrum has been computed and plotted. The plot has been saved as 'cmb_temperature_spectrum.png'. If you have any further questions or need additional assistance, feel free to ask!


--------------------------------------------------------------------------------

Calling: admin...



Provide feedback to chat_manager. Type 'proceed' to continue as suggested or type your suggestion, or type 'exit' to end the conversation:  exit


Number of reviews: 3
Reviews remaining: 0
Final Plan for the task:
1. **Retrieve Information**: 
   - Assign a classy_sz_agent to gather detailed information on computing CMB temperature power spectra using the `classy_sz` library.
   - Ensure the agent provides specific details on the necessary cosmological parameters and any relevant settings.

2. **Write Python Code**: 
   - Assign an engineer to write the Python code to perform the computation.
   - Include error handling and logging to facilitate debugging and verification.
   - Ensure the code computes only the temperature power spectra and excludes polarization spectra.

3. **Execute and Validate the Code**: 
   - Assign an executor to run the Python code.
   - Validate the output against expected results to ensure accuracy.

4. **Plot the Results**: 
   - Produce a plot of the CMB temperature power spectra after executing the code.

5. **Document and Review**: 
   - Have the engineer document the code and the process for future

In [12]:
context_variables

{'plans': ["To compute the CMB temperature power spectra using the `classy_sz` library without plotting the polarization power spectra, follow these steps:\n\n1. **Install the `classy_sz` Library**: Ensure that you have the `classy_sz` library installed. You can install it using pip if it's not already installed.\n\n   ```bash\n   pip install classy_sz\n   ```\n\n2. **Set Up the Cosmological Parameters**: Define the cosmological parameters required for the computation. These parameters include the Hubble constant, matter density, baryon density, etc.\n\n3. **Initialize the Class**: Create an instance of the `Class` from the `classy_sz` library.\n\n4. **Compute the Power Spectra**: Use the `compute()` method to calculate the power spectra. Make sure to specify that you only want the temperature power spectra.\n\n5. **Extract the Results**: Retrieve the temperature power spectra from the results.\n\n6. **Avoid Plotting Polarization**: Ensure that the settings or parameters used do not in